<div style='float: right'><img src='pic/kakkuro.png'/></div>
## <div id='kakkuro' />カックロ

In [ ]:
import numpy as np
from pulp import *
from ortoolpy import addvar, addvars, addbinvar, addbinvars
data = """\
******
**..**
*....*
*..*..
**....
***..*""".split()
hint = [[[int(s) if s else -1 for s in (t+'|').split('|')]
        for t in u.split(',')] for u in """\
,,11,4,,
,14|5,,,10,
|17,,,,,3
|6,,,3|4,,
,|10,,,,
,,|3,,,""".split()]

### 問題
* マスに 1～9 の数字を入れます
* 縦または横に連続する空マスの中に同じ数字は入りません
* 連続するマスの合計が示されます

### 定式化
\begin{array}{cl}
            変数 & v_{ijk} \in \{0, 1\} ~ \forall i, j, k ~ ~ ~ マスi,jが数字k+1か (1) \\
                 & r_{ij} \in Z ~ \forall i, j ~ ~ ~ ~ ~ マスi,jの数字 (2) \\
\mbox{subject to} & \sum_k{v_{ijk}} = 1 ~ \forall i, j ~ ~ ~ ~ 数字は1つ (3) \\
                 & \sum_k{k \times v_{ijk}} + 1 = r_{ij} ~ \forall i, j ~ ~ ~ rをvで表現 (4) \\
                 & \sum_{ij}{r_{ij}} = 合計 ~ \forall i, j ~ ~ ~ ~ ~ マスの合計が同じ (5) \\
                 & マスの並びで同じ数字は禁止 (6) \\
\end{array}

In [ ]:
nh, nw =len(data), len(data[0]) # 縦、横 
m = LpProblem()
v = np.array(addbinvars(nh, nw, 9)) # (1)
r = np.array(addvars(nh, nw)) # (2)
for i in range(nh):
    for j in range(nw):
        if data[i][j] != '*':
            m += lpSum(v[i,j,:]) == 1 # (3)
            m += lpDot(range(9), v[i,j,:]) + 1 == r[i,j] # (4)
            continue
        for drc in range(2): # 縦と横
            if hint[i][j][drc] < 0:
                continue
            y, x = i, j
            l = []
            while True:
                y, x = y + 1 - drc, x + drc
                if x == nw or y == nh or data[y][x] == '*':
                    break
                l.append((y,x))
            m += lpSum(r[y,x] for y,x in l) == hint[i][j][drc] # 合計 (5)
            for k in range(9):
                m += lpSum(v[y,x,k] for y,x in l) <= 1 # 並びで同じ数は1つ (6)
%time m.solve()
np.vectorize(lambda i: str(int(value(i))) if value(i) else ' ')(r).tolist()